In [11]:
from pearl.pearl_initial_formulation import *
from datastructures.graphutils import flatten_tree
from datastructures.api import edges_no_param, addobj, addineq, OPT, SolverRef, merge
from copy import deepcopy

In [2]:
#nodetyperepr = {VAR: '{}', COMP: 'f_{{{}}}', SOLVER: '{}'}
edges, tree = model.generate_formulation()

In [ ]:
edges_for_solving = copy_dicts(edges_nopar)
eqnidxs = list(edges_for_solving[1].keys())
varidxs = all_variables(*edges_for_solving)
graph_edges_minassign = all_edges(*edges_for_solving)
edges_left_right = list(dir_graph(graph_edges_minassign, eqnidxs, {}))

In [ ]:
not_input = [r'\omega_0', 'E_{move}', 'A_{33}', 'B_M', 'K_G', 'I', 'E_{recharge}', 'm_{struct}']
not_output = ['D_f', 'D_d', 'D_s', 't_s', 't_f', 't_d', 'P_{comms}', 'R'] #R
xsol,_ = min_arc_set_assign(min_assign_edges, varidxs, eqnidxs, not_input, not_output)

In [ ]:
selected = tuple((right, left) for left, right in edges_left_right if (left,right) in min_assign_edges and xsol[left, right] > 0.5)

In [3]:
opt = addsolver(m, name="opt", idbyname=True)
addobj(opt, mtot) # for optimization make sure to add OPT in model
addineq(opt, -td)
addineq(opt, hf-0.9*tf)
addineq(opt, Ds-0.9*Df)
addineq(opt, Ds-0.9*Dd)
addineq(opt, Ds-0.9*Dd)
addineq(opt, 0.1-P_move)
setsolvefor(opt, [tf, ts, Ds, Dd, hf], {tf:[0,10], ts:[0,10], Ds:[0,10], Dd:[0,10], hf:[0,10]})

In [5]:
edges_opt, tree_opt, comps_opt = merge(model, edges, tree)

In [6]:
edges_opt_nopar = edges_no_param(model, edges_opt) # filter out variables that are parameters

In [9]:
tree_opt = flatten_tree(tree, 'opt')

In [ ]:
prob, mdao_in, groups, namingfunc = model_to_problem(model, formulation=(edges, tree))